# Treinamento Multi-Modelo (Estratégia Delta)

**Contexto:**
Diferentes carteiras de crédito reagem de forma diferente à economia. A inadimplência Rural depende de safra; a PJ depende do PIB; a PF depende do desemprego. Por isso, treinaremos **modelos especialistas** para cada segmento.

**Estratégia Matemática (Delta Learning):**
Em vez de tentar prever o valor absoluto (ex: Inadimplência será 3.5%), vamos treinar o modelo para prever a **VARIAÇÃO (Delta)** em relação ao mês anterior.

$$\text{Previsão}_{t} = \text{Valor Real}_{t-1} + \Delta_{\text{Predito}}$$

Isso torna o sistema muito mais sensível a mudanças de tendência de curto prazo e corrige o viés de escala.

In [1]:
# --- 1. Importação e Configuração ---
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Configuração Visual
sns.set_theme(style="whitegrid")

# Definição de Caminhos (Pathlib garante compatibilidade Windows/Mac/Linux)
path_data = Path("../data/processed/df_modelagem_v3.csv")
path_models = Path("../models/")
path_models.mkdir(parents=True, exist_ok=True) # Cria a pasta models se não existir

# Carga dos Dados
if path_data.exists():
    df = pd.read_csv(path_data, index_col="data", parse_dates=True)
    print(f"✅ Base de dados carregada com sucesso. Shape: {df.shape}")
else:
    raise FileNotFoundError("ERRO CRÍTICO: Arquivo 'df_modelagem_v3.csv' não encontrado. Execute o notebook 03 primeiro.")

✅ Base de dados carregada com sucesso. Shape: (120, 50)


# Definição dos Alvos (Targets)

In [2]:
# --- 2. Definição dos Segmentos ---

# Mapeamento: "Nome Amigável do Modelo" -> "Coluna no DataFrame"
targets_potenciais = {
    "PF": "inad_pf_tot",
    "PJ": "inad_pj_tot",
    "Rural_PF": "inad_rd_pf_cr_rur_tot",  
    "Rural_PJ": "inad_rd_pj_cr_rur_tot"   
}

targets = {k: v for k, v in targets_potenciais.items() if v in df.columns}

print(f"Modelos que serão treinados: {list(targets.keys())}")

if len(targets) < len(targets_potenciais):
    print("Aviso: Algumas colunas de inadimplência rural não foram encontradas e serão ignoradas.")

Modelos que serão treinados: ['PF', 'PJ', 'Rural_PF', 'Rural_PJ']


## Pipeline de Treinamento em Loop

Para cada segmento (PF, PJ, Rural), o código abaixo vai:
1.  **Calcular o Delta:** Criar a variável alvo (Diferença mês a mês).
2.  **Isolar Variáveis:** Remover vazamento de dados (outras inadimplências).
3.  **Feature Engineering:** Adicionar o valor do mês anterior como input (Inércia).
4.  **Treinar:** Random Forest Regressor.
5.  **Serializar:** Salvar Modelo, Scaler e Metadados para o App.

In [3]:
# --- 3. Loop de Treinamento e Exportação ---

print("--- INICIANDO PROCESSAMENTO EM LOTE ---")

cols_leakage_base = [c for c in df.columns if c.startswith("inad_")]

for nome_modelo, target_col in targets.items():
    print(f"\nTreinando Modelo: {nome_modelo} (Alvo: {target_col})...")
    
    # A. Preparação do Dataset Temporário
    df_train = df.copy()
    
    # B. Engenharia do Alvo (Target Engineering)
    df_train["target_delta"] = df_train[target_col].diff()
    
    # C. Engenharia de Features (Feature Engineering)
    df_train["valor_anterior"] = df_train[target_col].shift(1)
    df_train = df_train.dropna()
    
    # D. Definição de X (Features) e y (Target)
    y = df_train["target_delta"]
    
    # Lógica de Limpeza do X:
    X = df_train.drop(columns=cols_leakage_base + ["target_delta"])
    X["inad_anterior"] = df_train["valor_anterior"]
    
    # Garantia de Dummies
    X = pd.get_dummies(X, drop_first=True)
    
    # E. Padronização (Scaling)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # F. Treinamento (Fit)
    # Usamos max_depth=10 para evitar overfitting excessivo em dados ruidosos
    model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
    model.fit(X_scaled, y)
    
    # --- G. SALVAMENTO DOS ARTEFATOS ---
    
    # 1. O Modelo
    joblib.dump(model, path_models / f"model_{nome_modelo}_multimodelo.pkl")
    
    # 2. O Scaler
    joblib.dump(scaler, path_models / f"scaler_{nome_modelo}_multimodelo.pkl")
    
    # 3. Lista de Colunas
    pd.DataFrame(columns=X.columns).to_csv(path_models / f"columns_{nome_modelo}_multimodelo.csv", index=False)
    
    # 4. Metadados de "Ancoragem"
    ultimo_dado = {
        "data_referencia": df.index[-1],
        "valor_ultimo_real": df.iloc[-1][target_col],
        "X_ultimo_real": X.iloc[-1].to_dict()
    }
    joblib.dump(ultimo_dado, path_models / f"meta_{nome_modelo}.pkl")
    
    print(f"Modelo salvo! | Último Valor Real: {ultimo_dado['valor_ultimo_real']:.2f}% em {ultimo_dado['data_referencia'].date()}")

print("\n--- PROCESSO CONCLUÍDO ---")
print(f"Todos os arquivos foram salvos em: {path_models.absolute()}")

--- INICIANDO PROCESSAMENTO EM LOTE ---

Treinando Modelo: PF (Alvo: inad_pf_tot)...
Modelo salvo! | Último Valor Real: 3.54% em 2025-12-01

Treinando Modelo: PJ (Alvo: inad_pj_tot)...
Modelo salvo! | Último Valor Real: 2.03% em 2025-12-01

Treinando Modelo: Rural_PF (Alvo: inad_rd_pf_cr_rur_tot)...
Modelo salvo! | Último Valor Real: 2.28% em 2025-12-01

Treinando Modelo: Rural_PJ (Alvo: inad_rd_pj_cr_rur_tot)...
Modelo salvo! | Último Valor Real: 0.32% em 2025-12-01

--- PROCESSO CONCLUÍDO ---
Todos os arquivos foram salvos em: c:\Users\pedro\projeto_inadimplencia\notebooks\..\models


## Próximos Passos

Com a pasta `models/` criada com as informações que o Streamlit precisa:
1.  **`model_*.pkl`**: O cérebro que prevê a variação.
2.  **`scaler_*.pkl`**: A calculadora que ajusta a escala dos dados.
3.  **`meta_*.pkl`**: O ponto de partida (última inadimplência conhecida).

Próxima etapa: construir o arquivo `app_1.py`.